In [1]:
import pandas as pd
from sqlalchemy import create_engine
import sqlalchemy 
import pymysql
pymysql.install_as_MySQLdb()
from config import mysqlpassword 

In [2]:
players_csv = "Resources/players.csv"
players_df = pd.read_csv(players_csv, index_col = 0)
players_df.head()

,Player,height,weight,collage,born,birth_city,birth_state
0,Curly Armstrong,180.0,77.0,Indiana University,1918.0,NaN,NaN
1,Cliff Barker,188.0,83.0,University of Kentucky,1921.0,Yorktown,Indiana
2,Leo Barnhorst,193.0,86.0,University of Notre Dame,1924.0,NaN,NaN
3,Ed Bartels,196.0,88.0,North Carolina State University,1925.0,NaN,NaN
4,Ralph Beard,178.0,79.0,University of Kentucky,1927.0,Hardinsburg,Kentucky


In [3]:
# Create key
players_df['namekey'] = players_df['Player'].str.lower()
for s in [" ", "'", "-", ".", ","]:
    players_df['namekey'] = players_df['namekey'].str.replace(s,'')
players_df.set_index('namekey', verify_integrity=True, inplace=True)
players_df.head()

,Player,height,weight,collage,born,birth_city,birth_state
namekey,,,,,,,
curlyarmstrong,Curly Armstrong,180.0,77.0,Indiana University,1918.0,NaN,NaN
cliffbarker,Cliff Barker,188.0,83.0,University of Kentucky,1921.0,Yorktown,Indiana
leobarnhorst,Leo Barnhorst,193.0,86.0,University of Notre Dame,1924.0,NaN,NaN
edbartels,Ed Bartels,196.0,88.0,North Carolina State University,1925.0,NaN,NaN
ralphbeard,Ralph Beard,178.0,79.0,University of Kentucky,1927.0,Hardinsburg,Kentucky


In [4]:
players_df = players_df[['Player', 'height', 'weight', 'collage', 'birth_city', 'birth_state']]
players_df.head()

,Player,height,weight,collage,birth_city,birth_state
namekey,,,,,,
curlyarmstrong,Curly Armstrong,180.0,77.0,Indiana University,NaN,NaN
cliffbarker,Cliff Barker,188.0,83.0,University of Kentucky,Yorktown,Indiana
leobarnhorst,Leo Barnhorst,193.0,86.0,University of Notre Dame,NaN,NaN
edbartels,Ed Bartels,196.0,88.0,North Carolina State University,NaN,NaN
ralphbeard,Ralph Beard,178.0,79.0,University of Kentucky,Hardinsburg,Kentucky


In [5]:
players_df = players_df.rename(columns= {"collage": "college"})
players_df.head()
players_df.index

Index([     'curlyarmstrong',         'cliffbarker',        'leobarnhorst',
                 'edbartels',          'ralphbeard',           'geneberce',
              'charlieblack',          'nelsonbobb',      'jakebornheimer',
               'vinceboryla',
       ...
           'denzelvalentine',        'fredvanvleet', 'taureanwallerprince',
                'okarowhite',     'isaiahwhitehead',        'troywilliams',
               'kylewiltjer',    'stephenzimmerman',          'paulzipser',
                'ivicazubac'],
      dtype='object', name='namekey', length=3922)

In [15]:
# Create Database Connection
rds_connection_string = f"root:{mysqlpassword}@localhost/basketball_db"
engine = create_engine(f'mysql://{rds_connection_string}')

# Confirm tables
engine.table_names()

['players', 'salary']

In [16]:
# Load Dataframe into database
players_df.to_sql(name='players', con=engine, if_exists='replace', dtype={'namekey':sqlalchemy.types.VARCHAR(length=64)})

In [17]:
# Confirm data added succesfully
pd.read_sql_query('select * from players', con=engine).head()

,namekey,Player,height,weight,college,birth_city,birth_state
0,curlyarmstrong,Curly Armstrong,180.0,77.0,Indiana University,None,None
1,cliffbarker,Cliff Barker,188.0,83.0,University of Kentucky,Yorktown,Indiana
2,leobarnhorst,Leo Barnhorst,193.0,86.0,University of Notre Dame,None,None
3,edbartels,Ed Bartels,196.0,88.0,North Carolina State University,None,None
4,ralphbeard,Ralph Beard,178.0,79.0,University of Kentucky,Hardinsburg,Kentucky
